In [1]:
import os
from glob import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter


import pickle
import gc

from tqdm import tqdm
tqdm.pandas()

/home/voloch/.local/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, make_scorer

In [56]:
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier

In [4]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = './models'
UTILS = './utils'

## prepare data, test / training sets

In [5]:
data = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_train_target.csv'), parse_dates=['timestamp'])
data.head()

,session_id,client_pin,timestamp,multi_class_target
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen


In [6]:
def get_time_of_day(inp_hour):
    if (inp_hour >= 12) and (inp_hour < 18):
        return 'day'
    elif (inp_hour >= 6) and (inp_hour < 12):
        return 'morning'
    elif (inp_hour >= 18) and (inp_hour <= 23):
        return 'evening'
    else:
        return 'night'

In [7]:
%%time
data['dom']  = data.timestamp.apply(lambda x: x.day)
data['dow']  = data.timestamp.apply(lambda x: x.weekday())
data['hour'] = data.timestamp.apply(lambda x: x.hour)
data['tod']  = data.hour.apply(get_time_of_day)

data.head()

CPU times: user 44.1 s, sys: 811 ms, total: 44.9 s
Wall time: 43.3 s


,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen,15,0,14,day
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest,21,5,12,day
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement,24,4,18,evening
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen,15,6,19,evening
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen,20,3,7,morning


In [8]:
data.keys()[4:]

Index(['dom', 'dow', 'hour', 'tod'], dtype='object')

In [9]:
target = data['multi_class_target']

In [10]:
#lb_dom  = LabelBinarizer().fit(data['dom'])
#lb_dow  = LabelBinarizer().fit(data['dow'])
#lb_hour = LabelBinarizer().fit(data['hour'])
#lb_tod  = LabelBinarizer().fit(data['tod'])

In [11]:
#dom_features  = ['dom_'  + str(el) for el in lb_dom.classes_]
#dow_features  = ['dow_'  + str(el) for el in lb_dow.classes_]
#hour_features = ['hour_' + str(el) for el in lb_hour.classes_]
#tod_features  = ['tod_'  + str(el) for el in lb_tod.classes_]

In [12]:
#%%time
#dom_prep  = lb_dom.transform(data['dom'])
#dow_prep  = lb_dow.transform(data['dow'])
#hour_prep = lb_hour.transform(data['hour'])
#tod_prep =  lb_tod.transform(data['tod'])

#dom_prep.shape, dow_prep.shape, hour_prep.shape, tod_prep.shape

saving LabelBinarizer for using in prepare to submit part

In [13]:
#pickle.dump(lb_dom,  open((os.path.join(UTILS, 'lb_dom.pkl')),  'wb'))
#pickle.dump(lb_dow,  open((os.path.join(UTILS, 'lb_dow.pkl')),  'wb'))
#pickle.dump(lb_hour, open((os.path.join(UTILS, 'lb_hour.pkl')), 'wb'))
#pickle.dump(lb_tod,  open((os.path.join(UTILS, 'lb_tod.pkl')),  'wb'))

merge current data with statistics

In [52]:
client_freq_targ = pd.read_csv(os.path.join(DATA_OWN, 'client_freq.csv'))
client_diff_freq_dow = pd.read_csv(os.path.join(DATA_OWN, 'client_diff_freq_dow.csv'))
client_diff_freq_tod = pd.read_csv(os.path.join(DATA_OWN, 'client_diff_freq_tod.csv'))

In [15]:
client_freq_targ.keys()

Index(['client_pin', 'client_freq_main_screen', 'client_freq_statement',
       'client_freq_credit_info', 'client_freq_own_transfer',
       'client_freq_mobile_recharge', 'client_freq_phone_money_transfer',
       'client_freq_card2card_transfer', 'client_freq_chat',
       'client_freq_card_recharge', 'client_freq_invest'],
      dtype='object')

In [16]:
data = data.merge(client_freq_targ, how= 'left', on='client_pin', validate='many_to_one')

In [17]:
#client_freq_features = ['client_freq_main_screen', 'client_freq_statement',
#       'client_freq_credit_info', 'client_freq_own_transfer',
#       'client_freq_mobile_recharge', 'client_freq_phone_money_transfer',
#       'client_freq_card2card_transfer', 'client_freq_chat',
#       'client_freq_card_recharge', 'client_freq_invest']
client_freq_features = client_freq_targ.keys()[1:]

In [ ]:
new_df = pd.merge(A_df, B_df,  how='left', left_on='[A_c1,c2]', right_on = '[B_c1,c2]')

In [18]:
glob_diff_freq_dom  = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dom.csv'))
glob_diff_freq_dow  = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dow.csv'))
glob_diff_freq_hour = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_hour.csv'))
glob_diff_freq_tod  = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_tod.csv'))

In [19]:
glob_diff_freq_dom.columns  = ['dom_'  + el for el in glob_diff_freq_dom.keys()]
glob_diff_freq_dow.columns  = ['dow_'  + el for el in glob_diff_freq_dow.keys()]
glob_diff_freq_hour.columns = ['hour_' + el for el in glob_diff_freq_hour.keys()]
glob_diff_freq_tod.columns  = ['tod_'  + el for el in glob_diff_freq_tod.keys()]

In [20]:
glob_diff_freq_dom = glob_diff_freq_dom.rename(  columns={'dom_dom': 'dom'})
glob_diff_freq_dow = glob_diff_freq_dow.rename(  columns={'dow_dow': 'dow'})
glob_diff_freq_hour = glob_diff_freq_hour.rename(columns={'hour_hour': 'hour'})
glob_diff_freq_tod = glob_diff_freq_tod.rename(  columns={'tod_tod': 'tod'})

#glob_freq_dom.keys(), glob_freq_dow.keys(), glob_freq_hour.keys(), glob_freq_tod.keys(), 

In [21]:
dom_diff_freq_features  = [el for el in glob_diff_freq_dom.keys()[1:]]
dow_diff_freq_features  = [el for el in glob_diff_freq_dow.keys()[1:]]
hour_diff_freq_features = [el for el in glob_diff_freq_hour.keys()[1:]]
tod_diff_freq_features  = [el for el in glob_diff_freq_tod.keys()[1:]]

In [22]:
data.shape

(5065350, 18)

In [23]:
data = data.merge(glob_diff_freq_dom, how= 'left', on='dom', validate='many_to_one')
data.shape

(5065350, 28)

In [24]:
data = data.merge(glob_diff_freq_dow, how= 'left', on='dow', validate='many_to_one')
data.shape

(5065350, 38)

In [25]:
data = data.merge(glob_diff_freq_hour, how= 'left', on='hour', validate='many_to_one')
data.shape

(5065350, 48)

In [26]:
data = data.merge(glob_diff_freq_tod, how= 'left', on='tod', validate='many_to_one')
data.shape

(5065350, 58)

In [27]:
data.sample(10)

,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod,client_freq_main_screen,client_freq_statement,...,tod_diff_glob_freq_main_screen,tod_diff_glob_freq_statement,tod_diff_glob_freq_credit_info,tod_diff_glob_freq_own_transfer,tod_diff_glob_freq_mobile_recharge,tod_diff_glob_freq_phone_money_transfer,tod_diff_glob_freq_card2card_transfer,tod_diff_glob_freq_chat,tod_diff_glob_freq_card_recharge,tod_diff_glob_freq_invest
1642635,10cdd51f7b1b5cd443fc31095fccded2,7bc3293d0d9d743996be456f7fe0f8ec,2020-05-05 07:15:04,main_screen,5,1,7,morning,0.728155,0.032362,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184
1263355,5b17b19aaeff3a643909d2dd30ee3066,d2c319fdc55d4fe17f66384d1d33dcd5,2020-01-26 18:26:06,statement,26,6,18,evening,0.294118,0.388235,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
3969526,eaa436bb9ac7c3d22045ffd419bed1c2,e7be82099ec955416ce4c26d95b4a7c5,2020-07-19 11:14:56,main_screen,19,6,11,morning,0.689775,0.230503,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184
1604883,33eed529a16a8baacaf54aaca6a9aec3,5d1d937ef5f1774b038b6d08083d96b8,2020-06-23 15:37:15,main_screen,23,1,15,day,0.734177,0.113924,...,-0.006382,-0.003040,0.008693,-0.003441,0.010104,-0.003059,-0.002092,-0.000810,-0.001044,0.001070
3538142,9f3b63e0db8430f84867159aa3d89c3a,264623c97164cf7e92c4eee045b8c20d,2020-07-21 17:29:45,own_transfer,21,1,17,day,0.068282,0.286344,...,-0.006382,-0.003040,0.008693,-0.003441,0.010104,-0.003059,-0.002092,-0.000810,-0.001044,0.001070
984654,925533593d8fe3c3646fcbbdc12eb1ce,5c6244105f41f2ed3ae413b09246d178,2020-08-31 07:09:26,phone_money_transfer,31,0,7,morning,0.115789,0.168421,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184
387500,22f739e0dd17ef4a7ffb076cf39775d5,5aa858721854f5152e6afcf72cf7f841,2020-03-12 20:20:55,own_transfer,12,3,20,evening,0.246753,0.246753,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
4661123,104ba8b6ee820ca384bc4aa2c1047487,6380935485a90762a8614f82d5862b77,2020-04-27 23:10:17,statement,27,0,23,evening,0.423841,0.281457,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
3419992,cf02582b3733c4674d7d0b44a9818d55,20df32f11a71353b2cfa27cfd257d85d,2020-04-30 12:36:11,statement,30,3,12,day,0.065789,0.184211,...,-0.006382,-0.003040,0.008693,-0.003441,0.010104,-0.003059,-0.002092,-0.000810,-0.001044,0.001070
823613,9c8b37884e6e9b35b7e5f051ce8ffaf7,aea49410e6e294ec7ea3c7d43ea71c1c,2020-03-07 11:51:35,main_screen,7,5,11,morning,0.866667,0.066667,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184


In [28]:
%%time
X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.33, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

CPU times: user 5.82 s, sys: 230 ms, total: 6.05 s
Wall time: 6.34 s


((3393784, 58), (1671566, 58), (3393784,), (1671566,))

In [43]:
#f1 = make_scorer(f1_score , average='macro')

In [61]:
using_features = []
using_features.extend(client_freq_features)
using_features.extend(dom_diff_freq_features)
using_features.extend(dow_diff_freq_features)
using_features.extend(hour_diff_freq_features)
using_features.extend(tod_diff_freq_features)

print(len(using_features), using_features)

50 ['client_freq_main_screen', 'client_freq_statement', 'client_freq_credit_info', 'client_freq_own_transfer', 'client_freq_mobile_recharge', 'client_freq_phone_money_transfer', 'client_freq_card2card_transfer', 'client_freq_chat', 'client_freq_card_recharge', 'client_freq_invest', 'dom_diff_glob_freq_main_screen', 'dom_diff_glob_freq_statement', 'dom_diff_glob_freq_credit_info', 'dom_diff_glob_freq_own_transfer', 'dom_diff_glob_freq_mobile_recharge', 'dom_diff_glob_freq_phone_money_transfer', 'dom_diff_glob_freq_card2card_transfer', 'dom_diff_glob_freq_chat', 'dom_diff_glob_freq_card_recharge', 'dom_diff_glob_freq_invest', 'dow_diff_glob_freq_main_screen', 'dow_diff_glob_freq_statement', 'dow_diff_glob_freq_credit_info', 'dow_diff_glob_freq_own_transfer', 'dow_diff_glob_freq_mobile_recharge', 'dow_diff_glob_freq_phone_money_transfer', 'dow_diff_glob_freq_card2card_transfer', 'dow_diff_glob_freq_chat', 'dow_diff_glob_freq_card_recharge', 'dow_diff_glob_freq_invest', 'hour_diff_glob_fre

## check classifiers with check time of work

In [ ]:
#clf_sgd_log   = SGDClassifier(loss = 'log', class_weight='balanced', n_jobs=-1)

#clf_knn  = KNeighborsClassifier()
#clf_svc  = SVC()
clf_gaus = GaussianNB()

In [47]:
%%time
#clf_sgd_hinge = SGDClassifier(loss = 'hinge', class_weight='balanced', n_jobs=-1)
clf_sgd_hinge = SGDClassifier(loss = 'hinge', n_jobs=-1)
clf_sgd_hinge.fit(X_train[using_features], y_train)
pred_sgd_hinge = clf_sgd_hinge.predict(X_val[using_features])
print(len(set(pred_sgd_hinge)), set(pred_sgd_hinge))
print(f1_score(y_val, pred_sgd_hinge, average  = 'micro'))

{'main_screen', 'invest', 'card2card_transfer', 'card_recharge', 'chat', 'statement', 'credit_info', 'mobile_recharge', 'phone_money_transfer', 'own_transfer'}
0.5868694385982964
CPU times: user 1min 33s, sys: 4.09 s, total: 1min 37s
Wall time: 47.3 s


In [ ]:
%%time
#clf_sgd_perc  = SGDClassifier(loss = 'perceptron', class_weight='balanced', n_jobs=-1)
clf_sgd_perc  = SGDClassifier(loss = 'perceptron', n_jobs=-1)
clf_sgd_perc.fit(X_train[using_features], y_train)
pred_sgd_perc = clf_sgd_perc.predict(X_val[using_features])
print(len(set(pred_sgd_perc)), set(pred_sgd_perc))
print(f1_score(y_val, pred_sgd_perc, average  = 'micro'))

In [ ]:
%%time
# 9 min

clf_mlp  = MLPClassifier((200, 50), learning_rate = 'adaptive', activation='logistic', verbose = True)
clf_mlp.fit(X_train[using_features], y_train)
pred_mlp = clf_mlp.predict(X_val[using_features])
print(set(pred_mlp))
print(f1_score(y_val, pred_mlp, average  = 'micro'))

In [ ]:
%%time
# TOOO LONG

#clf_knn.fit(X_train[using_features], y_train)
#pred_knn = clf_knn.predict(X_val[using_features])
#print(set(pred_knn))
#print(f1_score(y_val, pred_knn, average  = 'micro'))

In [ ]:
%%time
# TOOO LONG

#clf_svc.fit(X_train[using_features], y_train)
#pred_svc = clf_svc.predict(X_val[using_features])
#print(set(pred_svc))
#print(f1_score(y_val, pred_svc, average  = 'micro'))

In [ ]:
%%time
# 24 min

clf_rf   = RandomForestClassifier(n_jobs = -1, verbose = 1) 
clf_rf.fit(X_train[using_features], y_train)
pred_rf = clf_rf.predict(X_val[using_features])
print(set(pred_rf))
print(f1_score(y_val, pred_rf, average  = 'micro'))

In [ ]:
%%time
clf_gaus.fit(X_train[using_features], y_train)
pred_gaus = clf_gaus.predict(X_val[using_features])
print(set(pred_gaus))
print(f1_score(y_val, pred_gaus, average  = 'micro'))

In [51]:
%%time


adaboost_inp_clf = DecisionTreeClassifier()
clf_ab   = AdaBoostClassifier(adaboost_inp_clf)
clf_ab.fit(X_train[using_features], y_train)
pred_ab = clf_ab.predict(X_val[using_features])
print(len(set(pred_ab)), set(pred_ab))
print(f1_score(y_val, pred_ab, average  = 'micro'))

{'main_screen', 'invest', 'card2card_transfer', 'card_recharge', 'chat', 'statement', 'credit_info', 'mobile_recharge', 'phone_money_transfer', 'own_transfer'}
0.5780519584629025
CPU times: user 1h 56min 44s, sys: 42.1 s, total: 1h 57min 26s
Wall time: 2h 47s


In [55]:
%%time

clf_lr = LogisticRegression(n_jobs = -1, verbose = 1)
clf_lr.fit(X_train[using_features], y_train)
pred_lr = clf_lr.predict(X_val[using_features])
print(len(set(pred_lr)), set(pred_lr))
print(f1_score(y_val, pred_lr, average  = 'micro'))

/home/voloch/Anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/voloch/Anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/voloch/Anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]{'main_screen', 'invest', 'card2card_transfer', 'card_recharge', 'chat', 'statement', 'credit_info', 'mobile_recharge', 'phone_money_transfer', 'own_transfer'}
0.5946699083374513
CPU times: user 10min 33s, sys: 11.6 s, total: 10min 45s
Wall time: 11min 26s


In [64]:
print(len(set(pred_lr)), set(pred_lr))

10 {'main_screen', 'invest', 'card2card_transfer', 'card_recharge', 'chat', 'statement', 'credit_info', 'mobile_recharge', 'phone_money_transfer', 'own_transfer'}


In [57]:
%%time

clf_lrsvc = LinearSVC() # loss = ‘hinge’
clf_lrsvc.fit(X_train[using_features], y_train)
pred_lrsvc = clf_lrsvc.predict(X_val[using_features])
print(len(set(pred_lrsvc)), set(pred_lrsvc))
print(f1_score(y_val, pred_lrsvc, average  = 'micro'))

{'main_screen', 'invest', 'card2card_transfer', 'card_recharge', 'chat', 'statement', 'credit_info', 'mobile_recharge', 'phone_money_transfer', 'own_transfer'}
0.5954105312024772
CPU times: user 10min 39s, sys: 3.54 s, total: 10min 43s
Wall time: 11min 3s


In [60]:
%%time

clf_lrsvc_hinge = LinearSVC( loss = 'hinge')
clf_lrsvc_hinge.fit(X_train[using_features], y_train)
pred_lrsvc_hinge = clf_lrsvc_hinge.predict(X_val[using_features])
print(len(set(pred_lrsvc_hinge)), set(pred_lrsvc_hinge))
print(f1_score(y_val, pred_lrsvc_hinge, average  = 'micro'))

/home/voloch/Anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'main_screen', 'invest', 'card2card_transfer', 'card_recharge', 'chat', 'statement', 'credit_info', 'mobile_recharge', 'phone_money_transfer', 'own_transfer'}
0.5753664527754214
CPU times: user 11min 34s, sys: 7.3 s, total: 11min 41s
Wall time: 13min 44s


In [ ]:
%%time
clf_cb = CatBoostClassifier()
clf_cb.fit(X_train[using_features], y_train)
pred_cb = clf_cb.predict(X_val[using_features])
#print(set(pred_cb))
print(f1_score(y_val, pred_cb, average  = 'micro'))

In [ ]:
pred_list = [pred_sgd, pred_mlp, pred_knn, pred_svc, pred_rf, pred_ab, pred_gaus, pred_cb]

In [ ]:
#for el in itertools.permutations(pred_list):
#    pred_all = []
#    pred_all = []

In [ ]:
def set_pred(idx):
    
    return Counter( \
                   pred_sgd[idx], \
                   pred_mlp[idx], \
                   pred_knn[idx], \
                   pred_svc[idx], \
                   pred_rf[idx], \
                   pred_ab[idx], \
                   pred_gaus[idx], \
                   pred_cb[idx], \
                  ).most_common()[0][0]

In [ ]:
pred_all = [set_pred(el) for el in range(len(y_val))]
print(f1_score(y_val, pred_all, average  = 'micro'))

## classifiers GridSearch

## study best models on full data

In [ ]:
clf_sgd = SGDClassifier(loss = 'hinge', n_jobs=-1)
#clf_sgd_log   = SGDClassifier(loss = 'log', n_jobs=-1)
#clf_sgd  = SGDClassifier(loss = 'perceptron', n_jobs=-1)

clf_mlp  = MLPClassifier()
#clf_knn  = KNeighborsClassifier()
clf_svc  = SVC()
clf_rf   = RandomForestClassifier() 
clf_ab   = AdaBoostClassifier()
clf_gaus = GaussianNB()

In [ ]:
%%time
clf_sgd.fit(data[using_features], target)

## save models

In [68]:
%%time
#pickle.dump(clf_sgd, open(os.path.join(MODELS, 'clf_sgd.pkl'), 'wb'))
#pickle.dump(clf_mlp, open(os.path.join(MODELS, 'clf_mlp.pkl'), 'wb'))
#pickle.dump(clf_knn, open(os.path.join(MODELS, 'clf_knn.pkl'), 'wb'))
#pickle.dump(clf_lr, open(os.path.join(MODELS, 'clf_lr.pkl'), 'wb'))
pickle.dump(clf_lrsvc, open(os.path.join(MODELS, 'clf_lrsvc.pkl'), 'wb'))
#pickle.dump(clf_svc, open(os.path.join(MODELS, 'clf_svc.pkl'), 'wb'))
#pickle.dump(clf_rf,  open(os.path.join(MODELS, 'clf_rf.pkl'),  'wb'))
#pickle.dump(clf_ab,  open(os.path.join(MODELS, 'clf_ab.pkl'),  'wb'))
#pickle.dump(clf_gaus,open(os.path.join(MODELS, 'clf_gaus.pkl'),'wb'))
            
#clf_cb.save_model(os.path.join(MODELS, 'clf_cb.cbm'), format='cbm')

CPU times: user 976 µs, sys: 9 µs, total: 985 µs
Wall time: 3 ms
